In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite.pcmci import PCMCI
#from tigramite.independence_tests import ParCorr, CMIknn
import pickle
from IPython.display import clear_output
import time
import tensorflow as tf
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
import os 
from os import listdir
from os.path import isfile, join
warnings.filterwarnings("ignore")
import sys
import scipy
from collections import Counter
np.random.seed(0)

In [2]:
#subject_list = ['DAP001(1)', 'DAP002(1)', 'DAP003(1)', 'DAP004(1)', 'DAP005(1)', 'DAP006(1)', 'DAP007(1)', 'DAP008(1)', 'DAP009(1)', 'DAP010(1)', 'DAP014(2)', 'DAP075(1)', 'DAP080(1)', 'DAP094(1)', 'DAP114(1)']
#subject_list = ['DMC032(1)', 'DMC051(1)', 'DMC052(1)']
subject_list = []
mypath = "../data/DAPHNE/respeck_AP"
for f in listdir(mypath):
    if isfile(join(mypath, f)):
        if not f.startswith('.'):
            subject_list.append(f[:9])

In [3]:
def predict_coughing(file_name):
    cohort = file_name[1:3]
    respeck = pd.read_csv(
        "../data/DAPHNE/respeck_{}/{}_respeck_raw.csv".format(cohort, file_name),
        infer_datetime_format=True, parse_dates=["timestamp"],
        index_col="timestamp")

    # Calculate acceleration magnitude
    respeck["magnitude"] = np.sqrt(respeck.x ** 2 + respeck.y ** 2 + respeck.z ** 2)
    input_data = np.array(respeck[["x", "y", "z", "magnitude"]], dtype=np.float32)
    
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path="model_magnitude.tflite")
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Load output array with the first 32 row  window
    interpreter.set_tensor(input_details[0]['index'], np.array([input_data[:32]]))
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    # output: predict as [coughing non-coughing movement]
    output = [output[0][0]] # probability of coughing
    #output = tf.math.argmax(output)

    start = 32
    end = 64
    
    # Append predictions to output array in 32 row windows
    while (end <= len(input_data)):
        
        # TFlite API
        interpreter.set_tensor(input_details[0]['index'], np.array([input_data[start:end]]))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        # Calculate class index of highest softmax probability
        prediction = output_data[0][0]
        #prediction = tf.math.argmax(output_data[0])
        output = np.append(output, [prediction], axis=0)
        
        start = end
        end = start + 32
        
    # Assign boolean value to coughing (True) or not coughing (False) value
    # output = output == 0
        
    # Group respeck data into 32 row windows
    df = respeck.reset_index()[["x", "y", "z", "timestamp"]]
    d = {'timestamp': 'last', 'x': 'mean', 'y': 'mean', 'z': 'mean'}
    df = df.groupby(df.index // 32).agg(d)
    
    # Assign coughing feature to respeck data
    df = df.iloc[:len(output)]
    df["coughing"] = output
    df.index = df.timestamp
    df.drop("timestamp", axis=1)
    
    return df

In [5]:
subject_list = ['DMC032(1)', 'DMC051(1)', 'DMC052(1)']

for sub in subject_list:
    df = predict_coughing(sub)[['coughing']]
    try:
        df.to_csv('../data/DAPHNE/coughing/{}.csv'.format(sub), index=True)
    except:
        print("Failed for {}.".format(sub))
        continue

In [53]:
df = predict_coughing('DAP014(2)')

In [54]:
df

,timestamp,x,y,z,coughing
timestamp,,,,,
2019-06-04 05:17:02.534,2019-06-04 05:17:02.534,-0.008018,0.021187,-0.998146,7.264254e-07
2019-06-04 05:17:05.094,2019-06-04 05:17:05.094,-0.008171,0.020744,-0.998184,7.793970e-07
2019-06-04 05:17:07.654,2019-06-04 05:17:07.654,-0.007759,0.021103,-0.997581,8.484522e-07
2019-06-04 05:17:10.214,2019-06-04 05:17:10.214,-0.007469,0.021095,-0.998108,9.438285e-07
2019-06-04 05:17:12.774,2019-06-04 05:17:12.774,-0.007317,0.019424,-0.992882,4.858306e-04
...,...,...,...,...,...
2019-06-06 05:46:47.793,2019-06-06 05:46:47.793,0.181229,-0.955544,0.205170,2.632042e-03
2019-06-06 05:46:50.677,2019-06-06 05:46:50.677,0.214066,-0.952805,0.165619,2.862835e-05
2019-06-06 05:46:53.432,2019-06-06 05:46:53.432,0.280182,-0.939331,0.193932,8.053604e-08
